<a href="#measuring-churn-rate">Measuring Churn Rate</a>
- <a href="#standard-based-churn-rate">Standard Churn Rate (Subscription)</a>
- <a href="#activity-based-churn-rate">Activity Churn Rate (Non-Subscription)</a>

<a href="#measuring-customers">Measuring Customers</a>
- <a href="#common-summay-metrics">Common Summary Metrics (Count, Total, Avg, etc)</a>
- <a href="#account-tenure">Account Tenure Metrics</a>

<a href="#observing-churn">Observing Churn</a>
- ..
- ..

In [666]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta


In [667]:
df = pd.read_json('data/mini_sparkify_event_data.json', orient='records', lines=True)
df.head(10)

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
0,1538352117000,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Martha Tilston,Rockpools,277.89016
1,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424
2,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730
3,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302
4,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771
5,1538352678000,9,8,NextSong,Logged In,PUT,200,free,81,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,The All-American Rejects,Don't Leave Me,208.29995
6,1538352886000,9,8,NextSong,Logged In,PUT,200,free,82,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,The Velvet Underground / Nico,Run Run Run,260.46649
7,1538352899000,30,29,NextSong,Logged In,PUT,200,paid,53,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Starflyer 59,Passengers (Old Album Version),185.44281
8,1538352905000,30,29,Add to Playlist,Logged In,PUT,200,paid,54,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,NaN,NaN,NaN
9,1538353084000,30,29,NextSong,Logged In,PUT,200,paid,55,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Frumpies,Fuck Kitty,134.47791


In [668]:
df = df.drop(np.where(df.userId == '')[0])
df.head(10)

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
0,1538352117000,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Martha Tilston,Rockpools,277.89016
1,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424
2,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730
3,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302
4,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771
5,1538352678000,9,8,NextSong,Logged In,PUT,200,free,81,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,The All-American Rejects,Don't Leave Me,208.29995
6,1538352886000,9,8,NextSong,Logged In,PUT,200,free,82,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,The Velvet Underground / Nico,Run Run Run,260.46649
7,1538352899000,30,29,NextSong,Logged In,PUT,200,paid,53,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Starflyer 59,Passengers (Old Album Version),185.44281
8,1538352905000,30,29,Add to Playlist,Logged In,PUT,200,paid,54,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,NaN,NaN,NaN
9,1538353084000,30,29,NextSong,Logged In,PUT,200,paid,55,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Frumpies,Fuck Kitty,134.47791


In [669]:
# Convert User Id to numeric type
df.userId = df.userId.astype('int')

In [670]:
# Drop 'useless' attributes

df = df.drop(['registration', 'userAgent', 'status', 'method', ], axis=1)
df.head(10)

,ts,userId,sessionId,page,auth,level,itemInSession,location,lastName,firstName,gender,artist,song,length
0,1538352117000,30,29,NextSong,Logged In,paid,50,"Bakersfield, CA",Freeman,Colin,M,Martha Tilston,Rockpools,277.89016
1,1538352180000,9,8,NextSong,Logged In,free,79,"Boston-Cambridge-Newton, MA-NH",Long,Micah,M,Five Iron Frenzy,Canada,236.09424
2,1538352394000,30,29,NextSong,Logged In,paid,51,"Bakersfield, CA",Freeman,Colin,M,Adam Lambert,Time For Miracles,282.82730
3,1538352416000,9,8,NextSong,Logged In,free,80,"Boston-Cambridge-Newton, MA-NH",Long,Micah,M,Enigma,Knocking On Forbidden Doors,262.71302
4,1538352676000,30,29,NextSong,Logged In,paid,52,"Bakersfield, CA",Freeman,Colin,M,Daft Punk,Harder Better Faster Stronger,223.60771
5,1538352678000,9,8,NextSong,Logged In,free,81,"Boston-Cambridge-Newton, MA-NH",Long,Micah,M,The All-American Rejects,Don't Leave Me,208.29995
6,1538352886000,9,8,NextSong,Logged In,free,82,"Boston-Cambridge-Newton, MA-NH",Long,Micah,M,The Velvet Underground / Nico,Run Run Run,260.46649
7,1538352899000,30,29,NextSong,Logged In,paid,53,"Bakersfield, CA",Freeman,Colin,M,Starflyer 59,Passengers (Old Album Version),185.44281
8,1538352905000,30,29,Add to Playlist,Logged In,paid,54,"Bakersfield, CA",Freeman,Colin,M,NaN,NaN,NaN
9,1538353084000,30,29,NextSong,Logged In,paid,55,"Bakersfield, CA",Freeman,Colin,M,Frumpies,Fuck Kitty,134.47791


In [671]:
# Join First Name and Last Name to Fullname column, then drop First Name, Last Name

df['fullname'] = df['firstName'] + ' ' + df['lastName']
df = df.drop(['firstName', 'lastName'], axis=1)

In [672]:
# Test datetime module convert from timestamp to datetime
datetime.fromtimestamp(1538352117).isoformat()

'2018-10-01T08:01:57'

In [673]:
# Create 'datetime' column, based on timpstamp value, and
# Convert to pandas datetime type

df['datetime'] = df['ts'].apply(lambda x: datetime.fromtimestamp(x/1000).isoformat())
df['datetime'] = df['datetime'].astype('datetime64[ns]')

In [674]:
# Check earlist date and latest date in the dataset

print(np.min(df['datetime']))
print(np.max(df['datetime']))

2018-10-01 08:01:57
2018-12-03 09:11:16


In [675]:
# Date range in the dataset
np.max(df['datetime']) - np.min(df['datetime'])

Timedelta('63 days 01:09:19')

In [676]:
# Helper Function. get timedelta
# Get a user's timerange of event. (delta of earlist event date to latest event date)

def get_timedelta_by_user(user_id, data=df):
    min_datetime = np.min(data[data.userId == user_id]['datetime'])
    max_datetime = np.max(data[data.userId == user_id]['datetime'])
    d = max_datetime - min_datetime
    return d

# Spot Check
get_timedelta_by_user(152)

Timedelta('31 days 17:01:23')

In [677]:
# Spot Check: Cancellation Event Count

df[(df.page == 'Cancel') | (df.page == 'Cancellation Confirmation')]['page'].value_counts()

Cancel                       52
Cancellation Confirmation    52
Name: page, dtype: int64

In [678]:
# Spot Check: Cancellation Event Count (string method)

df[(df.page.str.startswith('Cancel'))]['page'].value_counts()

Cancel                       52
Cancellation Confirmation    52
Name: page, dtype: int64

In [679]:
# Number of event records

df.shape[0]

278154

In [680]:
# Number of distinct users

df.userId.nunique()

225

In [681]:
# Number of 'authentication' values check

df['auth'].value_counts()

Logged In    278102
Cancelled        52
Name: auth, dtype: int64

---

<a id="measuring-churn-rate"></a>

## Measuring Churn Rate

---

observation date recall that follow weekly cycles, so pick a last sunday (or monday 0:00) date as end day

In [682]:
# Set the observation timeframe we want to do churn model.
# 4 Weeks. 

observation_end_date = pd.Timestamp('2018-12-03') # 49th Monday of 2018.
observation_start_date = observation_end_date - pd.Timedelta("28 days") # 45th Monday of 2018

In [683]:
def check_last_event(ids, data=df):
    # Check the last event for a list of users.
    
    # return (page, auth, level, time)
    
    l = []
    
    for i in ids:
        page = data[data.userId == i].tail(1).page.values[0]
        auth = data[data.userId == i].tail(1).auth.values[0]
        level = data[data.userId == i].tail(1).level.values[0]
        dt = pd.Timestamp(data[data.userId == i].tail(1).datetime.values[0])
        
        l.append((i, page, auth, level, dt))
        
    return l


---
<a id="standard-based-churn-rate"></a>

### Standard account-basd churn rate

Glance of churn rate for 2018 Nov full calendar.

Since the limitation of event data, we dont have access to Subscription database (information such as sub id, renew date, therefore, to utilize the event data, we calculate the standard account-based churn rate (for Nov 2018) as:

Number of users whose first event is a paid status last event is also a paid status and not cancel auth
/
Total distinct number of users in the month




In [684]:
# Define Nov start and end dates
nov_start_date = '2018-11-01T00:00:00'
nov_end_date = '2018-12-01T00:00:00'

# 2018 Nov Data Frame
df_nov = df[(df.datetime > nov_start_date) & (df.datetime < nov_end_date)]


# Distinct user ids in Nov 2018
nov_uids = df_nov.userId.value_counts().index 

# Numbers of user ids in Nov 2018
n_nov_uids = len(nov_uids) 

In [685]:
# The VERY FIRST event for each user in Nov 2018
df_nov_first = df_nov.drop_duplicates('userId', keep='first').\
sort_values('userId')[['userId', 'auth', 'level', 'datetime']]

df_nov_first = df_nov_first.set_index('userId')

In [686]:
# Users that first event in Nov 2018 is under 'paid' status. (Subscriber already)
df_nov_starts_with_paid = df_nov_first[df_nov_first['level'] == 'paid']
df_nov_starts_with_paid

,auth,level,datetime
userId,,,
2,Logged In,paid,2018-11-01 05:58:30
4,Logged In,paid,2018-11-04 21:23:21
6,Logged In,paid,2018-11-01 14:40:13
9,Logged In,paid,2018-11-01 09:49:01
10,Logged In,paid,2018-11-03 22:27:42
...,...,...,...
300020,Logged In,paid,2018-11-13 20:06:12
300021,Logged In,paid,2018-11-01 05:23:32
300022,Logged In,paid,2018-11-05 09:08:50


In [687]:
# The VERY LAST event for each user in Nov 2018

df_nov_last = df_nov.drop_duplicates('userId', keep='last').\
sort_values('userId')[['userId', 'auth', 'level', 'datetime']]

df_nov_last = df_nov_last.set_index('userId')
df_nov_last.head(10)

,auth,level,datetime
userId,,,
2,Logged In,paid,2018-11-22 06:30:04
4,Logged In,paid,2018-11-30 23:57:12
5,Logged In,free,2018-11-08 08:33:21
6,Logged In,paid,2018-11-30 06:48:26
7,Logged In,free,2018-11-23 14:46:51
8,Logged In,free,2018-11-30 21:54:57
9,Logged In,paid,2018-11-30 22:19:12
10,Logged In,paid,2018-11-19 20:49:48
11,Logged In,paid,2018-11-27 17:08:22


In [688]:
# Merger userFirst and userLast. Then check who churn under condition: 
# 1 - if begin with paid, end with free. (cancelled before the very last), OR
# 2 - if begin with paid, end with paid but it is ALSO the last paid event (cancel is the last event)

df_nov_first_last = pd.merge(df_nov_first, df_nov_last, how='inner', on='userId')

df_nov_churn = df_nov_first_last[
    ((df_nov_first_last.level_x == 'paid') & (df_nov_first_last.level_y == 'free')) | 
    ((df_nov_first_last.level_x == 'paid') & (df_nov_first_last.auth_y == 'Cancelled'))
]

df_nov_churn

,auth_x,level_x,datetime_x,auth_y,level_y,datetime_y
userId,,,,,,
28,Logged In,paid,2018-11-01 00:02:48,Cancelled,paid,2018-11-06 08:20:32
29,Logged In,paid,2018-11-01 00:00:27,Cancelled,paid,2018-11-14 15:37:11
35,Logged In,paid,2018-11-01 13:23:02,Logged In,free,2018-11-30 18:24:09
39,Logged In,paid,2018-11-01 00:35:46,Logged In,free,2018-11-30 13:27:16
53,Logged In,paid,2018-11-04 08:20:10,Cancelled,paid,2018-11-19 20:34:56
54,Logged In,paid,2018-11-01 00:01:56,Cancelled,paid,2018-11-13 03:40:08
70,Logged In,paid,2018-11-02 09:20:40,Cancelled,paid,2018-11-15 06:34:27
92,Logged In,paid,2018-11-01 00:00:06,Logged In,free,2018-11-30 23:58:18
103,Logged In,paid,2018-11-03 16:27:37,Cancelled,paid,2018-11-04 22:01:31


In [689]:
# Calculate Nov 2018 Churn Rate based on Standard account-based (paid-status or cancellation event)

print(f"Number of customers at the start of nov: {df_nov_starts_with_paid.shape[0]}")
print(f"Number of customers at the end of nov: {df_nov_churn.shape[0]}")

nov_churn_rate = df_nov_churn.shape[0] / df_nov_starts_with_paid.shape[0]

print(f"2018 Nov churn rate: {nov_churn_rate}")
print(f"2018 Nov retention rate: {1 - nov_churn_rate}")

Number of customers at the start of nov: 103
Number of customers at the end of nov: 24
2018 Nov churn rate: 0.23300970873786409
2018 Nov retention rate: 0.7669902912621359


---
<a id="activity-based-churn-rate"></a>
### Activity Based Churn for non-subscriber

In addtion to standard churn rate calculation, I would also calculate churn rate for non-paid ('free') user, based on activity
 measure for Nov 2018.
 
We can obtain an overview of the engagement, utilization of free user. 

How many of free user discontine to use the free service. 

Based on activity recency. Define an inactive time interval, eg 1 month.
Short gaps can be ignore



1. Observe on the start day. (Nov 1), get all free uses have activities within oct 1 - nov 1. Get the head count. 
2. Observe on the end day. (Before Dec 1), get all users have activities within nov 1 - dec 1. Get the head count. 
3. Calculation: End head count / Start head count  (Retaintion Rate)
4. Churn Rate + Retaintion Rate = 1

In [690]:
# Two observations: 
# 1. Observation on start date of the month 
# 2. Observation on end date of the month
# Recency interval: 30 days

nov_start_date = '2018-11-01T00:00:00'
nov_end_date = '2018-12-01T00:00:00'
recency_interval = '30 days'

# Convert to pd timestamp type
free_nov_ob_start_date = pd.Timestamp(nov_start_date)
free_nov_ob_end_date = pd.Timestamp(nov_end_date)

# Calcuate the inactivity limit for start and end
recency_start_date = free_nov_ob_start_date - pd.Timedelta(recency_interval)
recency_end_date = free_nov_ob_end_date - pd.Timedelta(recency_interval)

print(recency_start_date)
print(recency_end_date)

2018-10-02 00:00:00
2018-11-01 00:00:00


In [691]:
# Free users count at the start

free_start_count = \
df[
    (df.datetime > recency_start_date) & 
    (df.datetime < recency_end_date) & 
    (df.level == 'free')
].groupby('userId').size().shape[0]

free_start_count

175

In [692]:
# Free users count at the end

free_end_count = \
df[
    (df.datetime > recency_end_date) & 
    (df.datetime < nov_end_date) & 
    (df.level == 'free')
].groupby('userId').size().shape[0]

free_end_count # retain

103

In [693]:
print(f"Number of free users churn for 2018 Nov (Based on activity recency): {free_start_count - free_end_count}")

print(f"Free Users Churn Rate for 2018 Nov (Based on activity recency): \
{(free_start_count - free_end_count) / free_start_count}")

print(f"Free Users Retain Rate for 2018 Nov (Based on activity recency): \
{free_end_count / free_start_count}")

Number of free users churn for 2018 Nov (Based on activity recency): 72
Free Users Churn Rate for 2018 Nov (Based on activity recency): 0.4114285714285714
Free Users Retain Rate for 2018 Nov (Based on activity recency): 0.5885714285714285


---

<a id="measuring-customers"></a>

## Measuring Customers (Feature Engineering)


- Measuring counts, averages, and totals of customer events 
- Running QA tests on metrics 
- Choosing time periods and timestamps for metrics 
- Measuring how long a customer has been using a service 
- Measuring subscription metrics

---

In [694]:
# The measuring metrics based on this timeframe (4 weeks)

print(observation_start_date)  # 45th Monday of 2018
print(observation_end_date)  # 49th Monday of 2018.

# 4 weeks in between
observation_end_date - observation_start_date

2018-11-05 00:00:00
2018-12-03 00:00:00


Timedelta('28 days 00:00:00')

In [695]:
# Moving Periods. 
# 4 weeks for each period. Move one week each step.

periods = pd.date_range(start=observation_start_date - pd.Timedelta('21 days'), end=observation_end_date, freq='7D')
periods

DatetimeIndex(['2018-10-15', '2018-10-22', '2018-10-29', '2018-11-05',
               '2018-11-12', '2018-11-19', '2018-11-26', '2018-12-03'],
              dtype='datetime64[ns]', freq='7D')

In [696]:
# Softcopy for periods subset to work on.

df_p = df.copy()
df_p.head()

,ts,userId,sessionId,page,auth,level,itemInSession,location,gender,artist,song,length,fullname,datetime
0,1538352117000,30,29,NextSong,Logged In,paid,50,"Bakersfield, CA",M,Martha Tilston,Rockpools,277.89016,Colin Freeman,2018-10-01 08:01:57
1,1538352180000,9,8,NextSong,Logged In,free,79,"Boston-Cambridge-Newton, MA-NH",M,Five Iron Frenzy,Canada,236.09424,Micah Long,2018-10-01 08:03:00
2,1538352394000,30,29,NextSong,Logged In,paid,51,"Bakersfield, CA",M,Adam Lambert,Time For Miracles,282.82730,Colin Freeman,2018-10-01 08:06:34
3,1538352416000,9,8,NextSong,Logged In,free,80,"Boston-Cambridge-Newton, MA-NH",M,Enigma,Knocking On Forbidden Doors,262.71302,Micah Long,2018-10-01 08:06:56
4,1538352676000,30,29,NextSong,Logged In,paid,52,"Bakersfield, CA",M,Daft Punk,Harder Better Faster Stronger,223.60771,Colin Freeman,2018-10-01 08:11:16


In [697]:
# Utility Function: Assign period number. 

def period_from_datetime(x):
    """
    Assign each log a period number based on date. 
    Default 0.
    '4' is actually the latest 4 full weeks (28 days).
    `period 3` is 7-days before `period 4`. (28 days).
    `period 2` is 7-days before `period 3`. (28 days).
    `period 1` is 7-days before `period 2`. (28 days).
    
    Each period is 28 days, moving 7 days ahead from its previous period. 
    (moving 7 days)
    """
    period = 0 # default 0: out of window concerned. to filter out.
    
    if (x >= pd.Timestamp('2018-10-15')) & (x < pd.Timestamp('2018-11-12')): 
        period = 1
    
    if (x >= pd.Timestamp('2018-10-22')) & (x < pd.Timestamp('2018-11-19')):
        period = 2
    
    if (x >= pd.Timestamp('2018-10-29')) & (x < pd.Timestamp('2018-11-26')):
        period = 3
        
    if (x >= pd.Timestamp('2018-11-05')) & (x < pd.Timestamp('2018-12-03')):
        period = 4
        
    return period


# df_p[-20:]['datetime'].apply(period_from_datetime)

In [698]:
# Add a column of period, assign each event to a period base on its datetime.

df_p['period'] = df_p['datetime'].apply(period_from_datetime)

In [699]:
# p1,2,3,4 is what we concern. 0 is out of timeframe we don't consider. 

df_p[df_p.period.isin([1,2,3,4])].head(5)

,ts,userId,sessionId,page,auth,level,itemInSession,location,gender,artist,song,length,fullname,datetime,period
43037,1539532821000,54,782,NextSong,Logged In,paid,97,"Spokane-Spokane Valley, WA",F,The Red Jumpsuit Apparatus,Your Guardian Angel,320.75710,Alexi Warren,2018-10-15 00:00:21,1
43038,1539532831000,101,813,Downgrade,Logged In,paid,137,"Denver-Aurora-Lakewood, CO",M,NaN,NaN,NaN,Alex Hogan,2018-10-15 00:00:31,1
43039,1539532851000,25,827,Add to Playlist,Logged In,paid,90,"Tampa-St. Petersburg-Clearwater, FL",F,NaN,NaN,NaN,Madison Morales,2018-10-15 00:00:51,1
43040,1539532918000,143,552,NextSong,Logged In,free,15,"Virginia Beach-Norfolk-Newport News, VA-NC",F,Savatage,Drive,197.45914,Molly Harrison,2018-10-15 00:01:58,1
43041,1539532933000,25,827,NextSong,Logged In,paid,91,"Tampa-St. Petersburg-Clearwater, FL",F,The Strokes,The Modern Age,208.06485,Madison Morales,2018-10-15 00:02:13,1


In [700]:
# Spot Check: Distinct Events

df['page'].value_counts()

NextSong                     228108
Thumbs Up                     12551
Home                          10082
Add to Playlist                6526
Add Friend                     4277
Roll Advert                    3933
Logout                         3226
Thumbs Down                    2546
Downgrade                      2055
Settings                       1514
Help                           1454
Upgrade                         499
About                           495
Save Settings                   310
Error                           252
Submit Upgrade                  159
Submit Downgrade                 63
Cancel                           52
Cancellation Confirmation        52
Name: page, dtype: int64

<a id="common-summay-metrics"></a>
### Common Summary Metrics

e.g Count, Total, Avg, Sum

In [701]:
# Summary df for n_like within 4 moving period, each period is 4 weeks

df_n_like = df_p[df_p.period.isin([1,2,3,4])]
df_n_like = df_n_like[df_n_like.page == 'Thumbs Up']
ds_like_count = df_n_like.groupby(['userId', 'period'])['page'].count()

# turn groupby output to dataframe
df_n_like_count = pd.DataFrame(ds_like_count).reset_index()
df_n_like_count.columns = ['userId', 'period', 'n_like']
df_n_like_count.head(10)

,userId,period,n_like
0,2,2,1
1,2,3,3
2,2,4,8
3,3,2,10
4,3,3,3
5,4,1,5
6,4,2,10
7,4,3,20
8,4,4,60
9,5,2,5


In [702]:
# Summary df for n_songplay within 4 moving period, each period is 4 weeks

df_n_play = df_p[df_p.period.isin([1,2,3,4])]
df_n_play = df_n_play[df_n_play.page == 'NextSong']
ds_nplay_count = df_n_play.groupby(['userId', 'period'])['page'].count()

# turn groupby output to dataframe
df_n_play_count = pd.DataFrame(ds_nplay_count).reset_index()
df_n_play_count.columns = ['userId', 'period', 'n_songplay']
df_n_play_count.head(10)

,userId,period,n_songplay
0,2,2,50
1,2,3,99
2,2,4,247
3,3,2,154
4,3,3,38
5,4,1,102
6,4,2,294
7,4,3,478
8,4,4,1144
9,5,2,78


In [703]:
# Summary df for n_displike within 4 moving period, each period is 4 weeks

df_n_dislike = df_p[df_p.period.isin([1,2,3,4])]
df_n_dislike = df_n_dislike[df_n_dislike.page == 'Thumbs Down']
ds_ndislike_count = df_n_dislike.groupby(['userId', 'period'])['page'].count()

# turn groupby output to dataframe
df_n_dislike_count = pd.DataFrame(ds_ndislike_count).reset_index()
df_n_dislike_count.columns = ['userId', 'period', 'n_dislike']
df_n_dislike_count.head(10)

,userId,period,n_dislike
0,2,4,3
1,3,2,3
2,4,1,4
3,4,2,1
4,4,3,8
5,4,4,13
6,6,1,2
7,6,2,4
8,6,3,10
9,6,4,11


In [704]:
# Summary df for n_addplaylist within 4 moving period, each period is 4 weeks

df_n_addplaylist = df_p[df_p.period.isin([1,2,3,4])]
df_n_addplaylist = df_n_addplaylist[df_n_addplaylist.page == 'Add to Playlist']
ds_n_addplaylist = df_n_addplaylist.groupby(['userId', 'period'])['page'].count()

# turn groupby output to dataframe
df_n_addplaylist_count = pd.DataFrame(ds_n_addplaylist).reset_index()
df_n_addplaylist_count.columns = ['userId', 'period', 'n_addplaylist']
df_n_addplaylist_count.head(10)

,userId,period,n_addplaylist
0,2,3,1
1,2,4,7
2,3,2,3
3,3,3,1
4,4,1,4
5,4,2,7
6,4,3,10
7,4,4,38
8,5,2,4
9,5,4,2


In [705]:
# Summary df for n_addfriend within 4 moving period, each period is 4 weeks

df_n_addfriend = df_p[df_p.period.isin([1,2,3,4])]
df_n_addfriend = df_n_addfriend[df_n_addfriend.page == 'Add Friend']
ds_n_addfriend = df_n_addfriend.groupby(['userId', 'period'])['page'].count()

# turn groupby output to dataframe
df_n_addfriend_count = pd.DataFrame(ds_n_addfriend).reset_index()
df_n_addfriend_count.columns = ['userId', 'period', 'n_addfriend']
df_n_addfriend_count.head(10)

,userId,period,n_addfriend
0,2,2,2
1,2,3,4
2,2,4,7
3,3,2,1
4,4,2,5
5,4,3,9
6,4,4,31
7,5,2,2
8,5,4,1
9,6,1,1


In [706]:
# Summary df for n_adview within 4 moving period, each period is 4 weeks

df_n_adview = df_p[df_p.period.isin([1,2,3,4])]
df_n_adview = df_n_adview[df_n_adview.page == 'Roll Advert']
ds_n_adview = df_n_adview.groupby(['userId', 'period'])['page'].count()

# turn groupby output to dataframe
df_n_adview_count = pd.DataFrame(ds_n_adview).reset_index()
df_n_adview_count.columns = ['userId', 'period', 'n_adview']
df_n_adview_count.head(10)

,userId,period,n_adview
0,3,3,1
1,4,1,4
2,5,2,4
3,5,4,3
4,6,1,1
5,6,2,1
6,6,3,1
7,6,4,2
8,7,1,2
9,7,2,9


In [707]:
# Summary df for n_session within 4 moving period, each period is 4 weeks

df_n_session = df_p[df_p.period.isin([1,2,3,4])]

df_n_session = \
pd.DataFrame(df_n_session.groupby(['userId', 'period', 'sessionId']).size()).reset_index().drop([0], axis=1)

df_n_session_count = pd.DataFrame(df_n_session.groupby(['userId','period'])['sessionId'].count()).reset_index()
df_n_session_count.columns = ['userId', 'period', 'n_session']
df_n_session_count


,userId,period,n_session
0,2,2,1
1,2,3,2
2,2,4,3
3,3,2,2
4,3,3,1
...,...,...,...
613,300024,1,1
614,300025,1,2
615,300025,2,3
616,300025,3,1


In [708]:
# Summary df for n_itemsession within 4 moving period, each period is 4 weeks
# Total items in all sessions per period per user (represents usage, shoule be corelated to song play)

ds_n_itemsession = pd.DataFrame(
    df_p[df_p.period.isin([1,2,3,4])].groupby(['userId', 'period','sessionId'])['itemInSession'].count()
).reset_index().groupby(['userId', 'period'])['itemInSession'].sum()

df_n_itemsession_count = pd.DataFrame(ds_n_itemsession).reset_index()
df_n_itemsession_count.columns = ['userId', 'period', 'n_itemsession']

df_n_itemsession_count 


,userId,period,n_itemsession
0,2,2,55
1,2,3,118
2,2,4,298
3,3,2,183
4,3,3,47
...,...,...,...
613,300024,1,102
614,300025,1,191
615,300025,2,139
616,300025,3,185


In [709]:
# Summary. Sessions Count, Session Items Count, Items per session each period. 

df_session_summary = pd.merge(df_n_session_count, df_n_itemsession_count)
df_session_summary['items_per_session'] = df_session_summary['n_itemsession'] / df_session_summary['n_session']
df_session_summary

,userId,period,n_session,n_itemsession,items_per_session
0,2,2,1,55,55.000000
1,2,3,2,118,59.000000
2,2,4,3,298,99.333333
3,3,2,2,183,91.500000
4,3,3,1,47,47.000000
...,...,...,...,...,...
613,300024,1,1,102,102.000000
614,300025,1,2,191,95.500000
615,300025,2,3,139,46.333333
616,300025,3,1,185,185.000000


In [710]:
# Summary. Total listen duration for each user per period. 

df_duration = df_p[(df_p['page'] == 'NextSong') & df_p.period.isin([1,2,3,4])][['userId', 'period', 'length']]

df_duration_total = pd.DataFrame(
    df_duration.groupby(['userId', 'period'])['length'].sum()
).reset_index()

df_duration_total.columns = ['userId', 'period', 'total_duration']
df_duration_total


,userId,period,total_duration
0,2,2,11945.95611
1,2,3,23915.42291
2,2,4,62124.84475
3,3,2,38639.29017
4,3,3,10421.45543
...,...,...,...
612,300024,1,22698.64417
613,300025,1,36197.72979
614,300025,2,27444.63957
615,300025,3,39919.26219


In [711]:
# Summary. Playback duration per session.

df_duration_m = \
df_p[(df_p['page'] == 'NextSong') & df_p.period.isin([1,2,3,4])][['userId','period','sessionId','length']]

df_duration_n_ses = pd.DataFrame(
    pd.DataFrame(
        df_duration_m.groupby(['userId', 'period', 'sessionId'])['length'].sum()
    ).reset_index()\
        .groupby(['userId', 'period'])['sessionId'].size()
        ).reset_index()

df_duration_total = pd.DataFrame(
    df_duration_m.groupby(['userId', 'period'])['length'].sum()
).reset_index()

df_duration_summary = pd.merge(df_duration_n_ses, df_duration_total)
df_duration_summary['avg_play_duration'] = df_duration_summary['length'] / df_duration_summary['sessionId']
df_duration_summary.columns = ['userId','period','n_play_session', 'total_playback_duration', 'avg_playback_duration']
df_duration_summary


,userId,period,n_play_session,total_playback_duration,avg_playback_duration
0,2,2,1,11945.95611,11945.956110
1,2,3,2,23915.42291,11957.711455
2,2,4,3,62124.84475,20708.281583
3,3,2,2,38639.29017,19319.645085
4,3,3,1,10421.45543,10421.455430
...,...,...,...,...,...
612,300024,1,1,22698.64417,22698.644170
613,300025,1,2,36197.72979,18098.864895
614,300025,2,3,27444.63957,9148.213190
615,300025,3,1,39919.26219,39919.262190


<a id="account-tenure"></a>
### Account Tenure

tenure should measure only the current subscription.



In [712]:
user4 = df[df.userId == 4]

In [713]:
np.where(user4.groupby('level').cumcount() == 0)

(array([  0, 141]),)

In [714]:
# experiment cumcount() shift()
# https://stackoverflow.com/questions/25119524/pandas-conditional-rolling-count

dft = pd.DataFrame({
    'x': [1,2,3,4,5,6,7,8,9,10,11],
    'y': ['a','a', 'a', 'b', 'b','b','b','b','a','a','a']
})

dft.groupby(
    (dft['y'] != dft['y'].shift(1)).cumsum()
).cumcount()

0     0
1     1
2     2
3     0
4     1
5     2
6     3
7     4
8     0
9     1
10    2
dtype: int64

In [715]:
# Experiment on Tenure calculation with User ID 4

user4['cumcount'] = user4.groupby(
    (user4['level'] != user4['level'].shift(1)).cumsum()
).cumcount()



/var/folders/z2/4x1xtsmd40sfmyp2ftrfbkfh0000gn/T/ipykernel_25639/1498505545.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user4['cumcount'] = user4.groupby(


In [716]:
user4[['level', 'cumcount', 'datetime']]

# Start Date of latest subscription
user4[(user4.cumcount == 0) & (user4.level == 'paid')].tail(1).iloc[0].datetime

# End Date of latest subscription
user4[
    (user4.level == 'paid') & (~user4.page.str.startswith('Cancel'))
].tail(1).iloc[0].datetime

# Tenure Days
(user4[
    (user4.level == 'paid') & (~user4.page.str.startswith('Cancel'))
].tail(1).iloc[0].datetime) - (user4[(user4.cumcount == 0) & (user4.level == 'paid')].tail(1).iloc[0].datetime)

Timedelta('40 days 05:03:25')

In [717]:
# Users in Observation timeframe [Timestamp('2018-11-05 00:00:00') to Timestamp('2018-12-03 00:00:00')]
# that also have 'paid' status. (subscriber)
# These are the users we concern currently.

uids_obs = list(df_p[
        (df_p.period == 4) & 
        (df_p.level == 'paid')
    ].groupby('userId').size().index)

len(uids_obs)

132

In [718]:
# Utility Function. Get tunure days
def get_tenures(ids=uids_obs):
     
    """
    Get user's tenure days.
    Key value pair.
    
    Input: list of user ids. Assuming that argument ids are paid user. 
    Output: dict of key value pairs. key: user, value: tunure
    """
    
    dfsub = df[df.datetime < observation_end_date]
    
    tenures = {}
    
    for i in ids:
        dfu = dfsub[dfsub.userId == i]
        
        # Make a cumulative count column
        dfu['cumcount'] = dfu.groupby(
            (dfu['level'] != dfu['level'].shift(1)).cumsum()
        ).cumcount()
        
        # Start Date of latest subscription
        start = dfu[(dfu.cumcount == 0) & (dfu.level == 'paid')].tail(1).iloc[0].datetime
        
        # End Date of latest subscription
        end = dfu[
            (dfu.level == 'paid') & (~dfu.page.str.startswith('Cancel'))
        ].tail(1).iloc[0].datetime
        
        
        # append result, first day starts with 0. plus 1 make it represents the first days more intuitive. 
        tenures[i] = (end - start).days + 1
        
    return tenures






In [719]:
tenures = get_tenures()
tenures

/var/folders/z2/4x1xtsmd40sfmyp2ftrfbkfh0000gn/T/ipykernel_25639/4267183752.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfu['cumcount'] = dfu.groupby(


{2: 45,
 4: 41,
 6: 55,
 9: 31,
 10: 43,
 11: 11,
 13: 1,
 14: 50,
 15: 55,
 16: 57,
 20: 11,
 21: 34,
 23: 2,
 25: 10,
 26: 1,
 28: 8,
 29: 37,
 30: 3,
 33: 58,
 35: 13,
 36: 18,
 37: 44,
 38: 37,
 39: 1,
 40: 31,
 41: 52,
 42: 55,
 45: 40,
 46: 29,
 49: 1,
 50: 9,
 52: 27,
 53: 26,
 54: 15,
 55: 1,
 56: 23,
 59: 25,
 60: 46,
 61: 26,
 62: 50,
 65: 54,
 66: 36,
 67: 32,
 69: 47,
 70: 43,
 71: 27,
 74: 28,
 75: 53,
 77: 36,
 79: 15,
 81: 43,
 82: 24,
 83: 24,
 85: 33,
 86: 16,
 88: 35,
 89: 23,
 91: 36,
 92: 47,
 93: 4,
 96: 4,
 97: 32,
 98: 36,
 99: 24,
 100: 16,
 104: 37,
 108: 24,
 109: 37,
 111: 27,
 113: 58,
 114: 30,
 115: 54,
 118: 49,
 120: 60,
 124: 60,
 126: 42,
 127: 57,
 128: 46,
 131: 50,
 132: 51,
 136: 45,
 137: 1,
 138: 40,
 139: 21,
 140: 11,
 141: 42,
 142: 61,
 145: 58,
 147: 17,
 151: 1,
 152: 15,
 153: 22,
 155: 20,
 100002: 38,
 100004: 2,
 100007: 59,
 100008: 43,
 100014: 42,
 100015: 23,
 100016: 54,
 100018: 38,
 200002: 29,
 200003: 4,
 200004: 56,
 200005: 2

In [720]:
tenures[25]

10

In [721]:
df[df.userId == 100]

,ts,userId,sessionId,page,auth,level,itemInSession,location,gender,artist,song,length,fullname,datetime
4582,1538492662000,100,256,NextSong,Logged In,paid,0,"Dallas-Fort Worth-Arlington, TX",M,Evanescence,Bring Me To Life,237.11302,Colin Larson,2018-10-02 23:04:22
4591,1538492899000,100,256,NextSong,Logged In,paid,1,"Dallas-Fort Worth-Arlington, TX",M,Sheena Easton,Strut (1993 Digital Remaster),239.62077,Colin Larson,2018-10-02 23:08:19
4592,1538492900000,100,256,Thumbs Down,Logged In,paid,2,"Dallas-Fort Worth-Arlington, TX",M,NaN,NaN,NaN,Colin Larson,2018-10-02 23:08:20
4596,1538493138000,100,256,NextSong,Logged In,paid,3,"Dallas-Fort Worth-Arlington, TX",M,Moonbootica (w/IAMX),Pretty Little Angel,407.27465,Colin Larson,2018-10-02 23:12:18
4607,1538493545000,100,256,NextSong,Logged In,paid,4,"Dallas-Fort Worth-Arlington, TX",M,tobyMac,City On Our Knees,272.09098,Colin Larson,2018-10-02 23:19:05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211954,1543586754000,100,2428,NextSong,Logged In,paid,88,"Dallas-Fort Worth-Arlington, TX",M,The White Stripes,Seven Nation Army,232.61995,Colin Larson,2018-11-30 22:05:54
211964,1543586986000,100,2428,NextSong,Logged In,paid,89,"Dallas-Fort Worth-Arlington, TX",M,Gillian Welch,Red Clay Halo,197.51138,Colin Larson,2018-11-30 22:09:46
211975,1543587183000,100,2428,NextSong,Logged In,paid,90,"Dallas-Fort Worth-Arlington, TX",M,Kid Cudi Vs Crookers,Day 'N' Nite,162.97751,Colin Larson,2018-11-30 22:13:03
211985,1543587345000,100,2428,NextSong,Logged In,paid,91,"Dallas-Fort Worth-Arlington, TX",M,Weezer,Island In The Sun,199.94077,Colin Larson,2018-11-30 22:15:45


---

<a id="observing-churn"></a>

## Observing Churn


- Picking a lead time in advance of churns for observation 
- Picking observation dates from subscriptions or activity 
- **Creating an analytic dataset by flattening metric data**
- Exporting a current customer list for segmentation

Forming Churn analysis dataset —A dataset in which every row represents a customer facing the decision to churn or stay. The outcome is what they do. The facts about these situations are the customer’s behavioral metric measurements (and possibly other data you have about them).

You want to observe them all at the same point (or points) **relative to their life cycle** with the product, which makes them comparable.






In [722]:
# Since we have not details access for every user's exact sub date (although, some can be inferenced from event, 
# evidence of transit from free to paid during timeframe. However, some users whose first event in log are already in 
# 'paid', means that are already ongoing sub we don't know when they sub).

# Therefore we won't have all sub/renew date for these 132 users. Not a good idea to user LeadTime which suggests
# observe at one-week before monthly renewal.
# For this case, event log only data, to make it consistent for known 132 users, I use Sequence of Regularly Spaced
# Observation dates that relative to each distinct user. 

# observation interval / length (21 days)
# frequency (observe every 14 days)
# intial outcome date (genesis from the last 'paid' event)
# iterate backwards

# note that the interval and freqency should be adjustable to experiment different results.

In [723]:
# Subset timeframe 

# Set the timeframe we want to do churn model.
# 4 Weeks. 

range_to = pd.Timestamp('2018-12-03') # 49th Monday of 2018.
range_from = range_to - pd.Timedelta("28 days") # 45th Monday of 2018

# Make a subset to extract user ids that we concern.
dfu = df[(df.datetime >= range_from) & (df.datetime < range_to)]

# get all paid user ids within this range (subscribers)
uids = dfu[dfu['level'] == 'paid'].drop_duplicates('userId', keep='last').sort_values(by='userId')['userId'].values
# print(len(uids))

# Make a dataframe only for these subscriber from orginal set.
dfsub = df[df.userId.isin(uids)]

uids_last_e = check_last_event(uids, data=dfu)
# uids_last_e

# Make a dataframe base on above
df_users_last_e = pd.DataFrame(
    {
        "userId": [x[0] for x in uids_last_e],
        "page": [x[1] for x in uids_last_e],
        "auth": [x[2] for x in uids_last_e],
        "level": [x[3] for x in uids_last_e],
        "datetime": [x[4] for x in uids_last_e],
    }
)

df_users_last_e

,userId,page,auth,level,datetime
0,2,Add Friend,Logged In,paid,2018-11-22 06:30:04
1,4,Add to Playlist,Logged In,paid,2018-12-01 00:32:28
2,6,Home,Logged In,paid,2018-11-30 06:48:26
3,9,Home,Logged In,paid,2018-11-30 22:19:12
4,10,Downgrade,Logged In,paid,2018-11-19 20:49:48
...,...,...,...,...,...
127,300020,NextSong,Logged In,paid,2018-11-13 22:15:34
128,300021,Home,Logged In,paid,2018-12-01 04:28:03
129,300022,Thumbs Up,Logged In,paid,2018-11-28 01:28:28
130,300023,Add to Playlist,Logged In,paid,2018-11-30 01:49:21


In [724]:
# Filter Out the outcome label.

# Condition:
# 1) if the e is 'free', means user trainsite from paid to free within the timeframe, OR
# 2) even tho e is 'paid', but its a 'Cancelled' auth, means the 'paid' is also churn. paid user submit cancel. 

# Churn users ids
u_churn = [x[0] for x in uids_last_e if (x[3] == 'free') or (x[2] == 'Cancelled')]


# Retain users
u_retain = [x[0] for x in uids_last_e if (x[3] == 'paid') and (x[2] != 'Cancelled')]

In [725]:
# Snippet for datetime

# Test datetime module convert from timestamp to datetime
datetime.fromtimestamp(1538352117).isoformat()
'2018-10-01T08:01:57'

# From isoformat to pandas datetime (pd Timestamp)
pd.to_datetime('2018-10-01T08:01:57')

# Time Delta object
pd.Timedelta('14 days')

# Calulation in time: Timestamp object +/- Timedelta object
pd.to_datetime('2018-10-01T08:01:57') - pd.Timedelta('14 days')

# datetime, timedelta
test_dt = datetime.fromtimestamp(df.loc[182974].ts/1000)
new_dt = test_dt + timedelta(days=9)
new_dt.isoformat()[:10]

Timestamp('2018-09-17 08:01:57')

In [726]:
# Get tenure daterange for a given user.
# method: latest seen 'paid' date minus tenure days.
# related method: get_tenure(). Already have a dict: tenures
# df[df.userId.isin(uids)]

u2_tenure_end = df[(df.userId == 2) & (df.level=='paid')].drop_duplicates('userId', keep='last').iloc[0].datetime

u2_tenure_start = u2_tenure_end-pd.Timedelta(f'{tenures[2]} days')

print(u2_tenure_start)


df[(df.userId == 2)]

2018-10-08 06:30:04


,ts,userId,sessionId,page,auth,level,itemInSession,location,gender,artist,song,length,fullname,datetime
21161,1538974195000,2,1,NextSong,Logged In,paid,0,"Raleigh, NC",F,Al Di Meola,Roller Jubilee,286.40608,Natalee Charles,2018-10-08 12:49:55
21172,1538974481000,2,1,NextSong,Logged In,paid,1,"Raleigh, NC",F,Karamelo Santo,El Baile Oficial,180.84526,Natalee Charles,2018-10-08 12:54:41
21179,1538974661000,2,1,NextSong,Logged In,paid,2,"Raleigh, NC",F,Hinder,Use Me,229.69424,Natalee Charles,2018-10-08 12:57:41
21186,1538974890000,2,1,NextSong,Logged In,paid,3,"Raleigh, NC",F,Milli Vanilli,Blame It On The Rain,248.52853,Natalee Charles,2018-10-08 13:01:30
21193,1538975138000,2,1,NextSong,Logged In,paid,4,"Raleigh, NC",F,The Killers,Human,360.75057,Natalee Charles,2018-10-08 13:05:38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182972,1542839400000,2,1928,About,Logged In,paid,102,"Raleigh, NC",F,NaN,NaN,NaN,Natalee Charles,2018-11-22 06:30:00
182973,1542839401000,2,1928,Add Friend,Logged In,paid,103,"Raleigh, NC",F,NaN,NaN,NaN,Natalee Charles,2018-11-22 06:30:01
182974,1542839402000,2,1928,Add Friend,Logged In,paid,104,"Raleigh, NC",F,NaN,NaN,NaN,Natalee Charles,2018-11-22 06:30:02
182975,1542839403000,2,1928,Add Friend,Logged In,paid,105,"Raleigh, NC",F,NaN,NaN,NaN,Natalee Charles,2018-11-22 06:30:03


In [727]:
# Dates Picking for users

# Define date picking constant parameter
frequency = pd.Timedelta('14 days')
interval = pd.Timedelta('14 days')

# Initial outcome and observation (snapshot) set up.
# (experimenting test for user id 2)

# ob_0
observation_date_initial = df_users_last_e[df_users_last_e.userId==2].iloc[0].datetime # to
observation_date_initial_from = observation_date_initial - interval # from
print(observation_date_initial)
print(observation_date_initial_from)
print()


# Next iteration. ob_1
observation_date_next = observation_date_initial - frequency # to
observation_date_next_from = observation_date_next - interval # from
print(observation_date_next)
print(observation_date_next_from)
print()


# Next next iteration
observation_date_next_next = observation_date_next - frequency # to
observation_date_next_next_from = observation_date_next_next - interval # from
print(observation_date_next_next)
print(observation_date_next_next_from)
print()



# We need to set boundary daterange for user within its tenure daterange to iterate. Call it tenure daterange.
# For every iteration of observation_date (the to date, not from), it must within the boundary


# Date Picking Algorithm

# For ongoing/retain users  (outcome always NOTCHURN, iteration observation outcome also NOTCHURN if iteration exists)
# 1) If the observation date is within tenure period so that the outcome label is retain
# It is guarentee to be retain since it is within his/her the tenure range, so that no worry if it has record
# on the date or nearest date
# AND also
# 2) If a observation date - user's first event date >= interval (means enough 14 days data)????


# For churn users  (intial outcome always CHURN, but next iteration observation(s) might not)
# 1) If the observation date is within tenure period so that the outcome label is retain
# It is guarentee to be retain since it is within his/her the tenure range, so that no worry if it has record
# on the date or nearest date
# AND also
# 2) If a observation date - user's first event date >= interval (means enough 14 days data)????

2018-11-22 06:30:04
2018-11-08 06:30:04

2018-11-08 06:30:04
2018-10-25 06:30:04

2018-10-25 06:30:04
2018-10-11 06:30:04



In [ ]:
# pseudo 

# For an ongoing/retain user: 
#
# observations = []
# intial ob_date = tenure_end_date
#
# while (ob_date isin tenure daterange), AND (ob_data - first_event_date >= interval)
#    ob_date_from = ob_date - interval
#    ob_churn_outcome = False
#    obervations.append([ob_data_from, ob_date, ob_churn_outcome])
#    ob_date -= frequency
#    
# ===================
# 
# For a churn user:
#
# observations = []
# initial ob_date = tenure_end_date
# initial iter_counter = 0
#
# while (ob_date isin tenure daterange), AND (ob_data - first_event_date >= interval)
#    ob_date_from = ob_date - interval
#    if counter == 0, then churn outcome is TRUE, else False
#    obervations.append([ob_data_from, ob_date, ob_churn_outcome])
#    ob_date -= frequency
#    iteration counter += 1

In [748]:
tenures

{2: 45,
 4: 41,
 6: 55,
 9: 31,
 10: 43,
 11: 11,
 13: 1,
 14: 50,
 15: 55,
 16: 57,
 20: 11,
 21: 34,
 23: 2,
 25: 10,
 26: 1,
 28: 8,
 29: 37,
 30: 3,
 33: 58,
 35: 13,
 36: 18,
 37: 44,
 38: 37,
 39: 1,
 40: 31,
 41: 52,
 42: 55,
 45: 40,
 46: 29,
 49: 1,
 50: 9,
 52: 27,
 53: 26,
 54: 15,
 55: 1,
 56: 23,
 59: 25,
 60: 46,
 61: 26,
 62: 50,
 65: 54,
 66: 36,
 67: 32,
 69: 47,
 70: 43,
 71: 27,
 74: 28,
 75: 53,
 77: 36,
 79: 15,
 81: 43,
 82: 24,
 83: 24,
 85: 33,
 86: 16,
 88: 35,
 89: 23,
 91: 36,
 92: 47,
 93: 4,
 96: 4,
 97: 32,
 98: 36,
 99: 24,
 100: 16,
 104: 37,
 108: 24,
 109: 37,
 111: 27,
 113: 58,
 114: 30,
 115: 54,
 118: 49,
 120: 60,
 124: 60,
 126: 42,
 127: 57,
 128: 46,
 131: 50,
 132: 51,
 136: 45,
 137: 1,
 138: 40,
 139: 21,
 140: 11,
 141: 42,
 142: 61,
 145: 58,
 147: 17,
 151: 1,
 152: 15,
 153: 22,
 155: 20,
 100002: 38,
 100004: 2,
 100007: 59,
 100008: 43,
 100014: 42,
 100015: 23,
 100016: 54,
 100018: 38,
 200002: 29,
 200003: 4,
 200004: 56,
 200005: 2

In [761]:
# Should remove outlier? newly sub. very limit event data
# maybe not. they still have at least one observation. its free usage will describe why become sub
# e.g 

df[(df.userId == 100004)]

,ts,userId,sessionId,page,auth,level,itemInSession,location,gender,artist,song,length,fullname,datetime
214694,1538534835000,100004,4,Home,Logged In,free,0,"Syracuse, NY",F,NaN,NaN,NaN,Emily Fisher,2018-10-03 10:47:15
214695,1538534867000,100004,4,NextSong,Logged In,free,1,"Syracuse, NY",F,John Powell,The Italian Job,108.32934,Emily Fisher,2018-10-03 10:47:47
214697,1538534975000,100004,4,NextSong,Logged In,free,2,"Syracuse, NY",F,Vienio & Pele,Automotywacja,38.34730,Emily Fisher,2018-10-03 10:49:35
214698,1538535013000,100004,4,NextSong,Logged In,free,3,"Syracuse, NY",F,3 Doors Down,These Days,219.61098,Emily Fisher,2018-10-03 10:50:13
214700,1538535232000,100004,4,NextSong,Logged In,free,4,"Syracuse, NY",F,The Trews,Fire Up Ahead,182.04689,Emily Fisher,2018-10-03 10:53:52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226354,1543458407000,100004,212,NextSong,Logged In,paid,117,"Syracuse, NY",F,Atreyu,Nevada's Grace (Album Version),227.99628,Emily Fisher,2018-11-29 10:26:47
226355,1543458634000,100004,212,NextSong,Logged In,paid,118,"Syracuse, NY",F,Dwight Yoakam,You're The One,239.30730,Emily Fisher,2018-11-29 10:30:34
226356,1543458649000,100004,212,Home,Logged In,paid,119,"Syracuse, NY",F,NaN,NaN,NaN,Emily Fisher,2018-11-29 10:30:49
226358,1543458873000,100004,212,NextSong,Logged In,paid,120,"Syracuse, NY",F,The Kooks,Shine On,192.96608,Emily Fisher,2018-11-29 10:34:33


In [747]:
# Snippet. Datetime

the_day_dt = datetime.fromtimestamp(df.loc[182974].ts/1000)

# 'on-the-day' plus one day and trim to 00:00, then can timeframe filter set condition before 
# the begining of next day, which also means taking all time on-the-day itself. 

the_day_plus1_dt = the_day_dt + timedelta(days=1)

# Trim hour/min/sec info, so default to 00:00
the_day_plus1_dt = the_day_plus1_dt.isoformat()[:10]

# Convert to pd.Timestamp
pd.Timestamp(new_dt.isoformat()[:10])


Timestamp('2018-11-23 00:00:00')